# Praca inżynierska
Marcin Bobiński
nr albumu: 297225

In [1]:
import os
HOME_PATH = os.getcwd()
RESEARCH_PATH = os.path.join(HOME_PATH, "models/research")
DATASET_PATH = os.path.join(HOME_PATH, "Dataset")
PRETRAINED_MODELS_PATH = os.path.join(HOME_PATH, "pretrained_models")
DATA_PATH = os.path.join(HOME_PATH, "data")
TF_RECORD_PATH = os.path.join(DATA_PATH, "tf_record")
MY_MODEL = os.path.join(HOME_PATH, "my_model")

# Przygotowanie
## TensorFlow Object Detection Api
Pierwszym krokiem jest sklonowanie repezytorium
```
git clone https://github.com/tensorflow/models.git
```
</br>

Następnie należy zainstalować przy pomocy menadżera pakietów pip lub w wirtualnym środowisku Docker. W przypadku wyboru pierwszej opcji zalecana jest instalacja na nowym środowisku.

1. <strong>Pip </strong> </br>
Najpierw należy zainstalować Protobufa np. https://github.com/protocolbuffers/protobuf/releases </br>
Oraz dodać go do ścieżki.
Następnie, przy jego pomocy z poziomu katalogu models/research wywołać:
```
protoc object_detection/protos/*.proto --python_out=.
```
Z tego samego poziomu skopiować plik instalacyjny:
```
cp object_detection/packages/tf2/setup.py .
```
A następnie dokonać instalacji:
```
python -m pip install --use-feature=2020-resolver .
```

2. <strong> Docker </strong></br>
Przygotowanie środowiska:
```
docker build -f research/object_detection/dockerfiles/tf2/Dockerfile -t od .
```
Uruchomienie środowiska:
```
docker run -it od
```

Test przeprowadzonej instalacji można wykonać przy pomocy komendy:
```
python object_detection/builders/model_builder_tf2_test.py
```

In [ ]:
# Instalacja PIP
os.chdir(RESEARCH_PATH)
os.system("protoc object_detection/protos/*.proto --python_out=.")
os.system("cp object_detection/packages/tf2/setup.py .")
os.system("python -m pip install --use-feature=2020-resolver .")

In [ ]:
# Instalacja Docker
os.chdir(HOME_PATH)
os.system("docker build -f research/object_detection/dockerfiles/tf2/Dockerfile -t od .")
os.system("docker run -it od")

In [5]:
# Weryfikacja poprawności instalacji
os.chdir(RESEARCH_PATH)
result = os.system("python object_detection/builders/model_builder_tf2_test.py")
assert result==0, "Object detection api wasn't installed properly"

## Pobranie pretrenowanych modeli

In [13]:
# Stworzenie folderu dla pretrenowanych modeli
os.chdir(HOME_PATH)
PRETRAINED_MODELS_PATH = os.path.join(HOME_PATH, "pretrained_models")
if not os.path.exists(PRETRAINED_MODELS_PATH):
    os.makedirs(PRETRAINED_MODELS_PATH)

### Mask Rcnn model

In [17]:
import tensorflow as tf
import shutil

name = "mask_rcnn"
base_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'
model_name =  "mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8"
model_file = model_name + '.tar.gz'

model_dir = tf.keras.utils.get_file(
    fname=model_name,
    origin=base_url + model_file,
    untar=True
)

MASK_RCNN_PATH = shutil.copytree(model_dir, os.path.join(PRETRAINED_MODELS_PATH, name))

## Pobranie datasetu
Zdjęcia drzew pobrane ze strony: http://labelme.csail.mit.edu/Release3.0/
</br>
Linki do pobrania datasetu:
- train: http://groups.csail.mit.edu/vision/LabelMe/Benchmarks/spain/training.tar.gz
- test: http://groups.csail.mit.edu/vision/LabelMe/Benchmarks/spain/test.tar.gz

In [4]:
import tensorflow as tf
import shutil

# Stworzenie folderu dla pobranego datasetu
os.chdir(HOME_PATH)
if not os.path.exists(DATASET_PATH):
    os.makedirs(DATASET_PATH)

train_url = "http://groups.csail.mit.edu/vision/LabelMe/Benchmarks/spain/training.tar.gz"
train_dir = tf.keras.utils.get_file(
    fname="train",
    origin=train_url,
    untar=True
)
_ = shutil.copytree(train_dir, os.path.join(DATASET_PATH, "train"))

test_url = "http://groups.csail.mit.edu/vision/LabelMe/Benchmarks/spain/test.tar.gz"
test_dir = tf.keras.utils.get_file(
    fname="test",
    origin=test_url,
    untar=True
)
_ = shutil.copytree(test_dir, os.path.join(DATASET_PATH, "test"))

1978990592/1978974639 [==============================] - 175s 0us/step


## Przygotowywanie zdjęć

In [99]:
import os
import shutil


_ = shutil.copytree(os.path.join(DATASET_PATH,"train"), os.path.join(DATA_PATH, "train"))
_ = shutil.copytree(os.path.join(DATASET_PATH,"test"), os.path.join(DATA_PATH, "test"))

In [100]:
from utils import prepare_data_utils as pdu

pdu.filterAndPrepareData(DATA_PATH, "train", ["tree"], ["trees"])
pdu.filterAndPrepareData(DATA_PATH, "test", ["tree"], ["trees"])

In [101]:
import os
import shutil
from utils import prepare_data_utils as pdu

pdu.flatten(os.path.join(DATA_PATH, "test"))
pdu.flatten(os.path.join(DATA_PATH, "train"))

shutil.rmtree(os.path.join(DATA_PATH, "train/Images" ))
shutil.rmtree(os.path.join(DATA_PATH, "train/Annotations" ))
shutil.rmtree(os.path.join(DATA_PATH, "test/Images" ))
shutil.rmtree(os.path.join(DATA_PATH, "test/Annotations" ))

# Uczenie modelu
## Przygotowanie plików tf record

In [3]:
import os

if not os.path.exists(TF_RECORD_PATH):
    os.makedirs(TF_RECORD_PATH)

In [4]:
from utils.tree_tf_record import create_tf_record_from_labelme_xml

create_tf_record_from_labelme_xml(os.path.join(DATA_PATH, "train"), os.path.join(TF_RECORD_PATH, "train"), 25)
create_tf_record_from_labelme_xml(os.path.join(DATA_PATH, "test"), os.path.join(TF_RECORD_PATH, "test"), 10)

Finished
Total annotations skipped: 53
Finished
Total annotations skipped: 46


Label Map

In [6]:
with open(os.path.join(DATA_PATH, "annotations.pbtxt"), 'w') as file:
    file.write("item { \n\tid: 1\n\tname: \'tree\'\n}")

Config

In [2]:
import os
import shutil

os.chdir(HOME_PATH)
if not os.path.exists(MY_MODEL):
    os.makedirs(MY_MODEL)

shutil.copyfile(
    os.path.join(PRETRAINED_MODELS_PATH,"mask_rcnn","pipeline.config"),
    MY_MODEL
)

PermissionError: [Errno 13] Permission denied: 'D:\\DEV\\GeoTree\\TreeDetection\\my_model'

Uczenie

In [3]:
print(HOME_PATH)

D:\DEV\GeoTree\TreeDetection


In [4]:

command = f"python object_detection/model_main_tf2.py \
    --pipeline_config_path={os.path.join(MY_MODEL,'pipeline.config')} \
    --model_dir={MY_MODEL} \
    --alsologtostderr"
print(command)

python object_detection/model_main_tf2.py     --pipeline_config_path=D:\DEV\GeoTree\TreeDetection\my_model\pipeline.config     --model_dir=D:\DEV\GeoTree\TreeDetection\my_model     --alsologtostderr


In [ ]:
os.chdir(RESEARCH_PATH)
os.system()